<a href="https://colab.research.google.com/github/kotaehyun/convert2Yolo/blob/main/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. YOLO**
* 이미지 분류, 객체 탐지, 인스턴스 분할 작업에 사용할 수 있는 모델
<center><img src='https://drive.google.com/uc?id=1yR--JjzSQ94_279aN2_UltkjzFMDPocB' width='400'></center>

* YOLO는 2015년 Joseph Redmond가 처음 출시한 이후 컴퓨터 비전 커뮤니티에 의해 성장
* 초기버전(1~4)에서의 YOLO는 Redmond가 작성한 커스텀 딥러닝 프레임워크인 Darknet에서 유지
* YOLOv3 레포를 PyTorch로 작성하여 Ultralytics에서 YOLOv5를 출시
* 유연한 Python 구조 덕분에 YOLOv5는 SOTA 레포가 되었음
* Ultralytics는 2023년 1월에 YOLOv8을 출시, 2024년 2월에 YOLOv9을 출시

#**2. 실습 데이터 준비**
* PascalVOC 2007
    * 분류와 객체 검출을 위해 만들어진 데이터셋
    * 객체 분할도 가능하지만, 더 최신 버전의 데이터셋을 사용하기 권장
    * 총 20개의 클래스를 가지고 있음
        * Person: person
        * Animal: bird, cat, cow, dog, horse, sheep
        * Vehicle: aeroplane, bicycle, boat, bus, car, motorbike, train
        * Indoor: bottle, chair, dining table, potted plant, sofa, tv/monitor
    * 학습 데이터
        * train: 2501장
        * val: 2510장
        * test: 4952장
    
> 학습 데이터가 너무 적어서 train과 val를 합쳐서 학습시킨 후, 데스트 데이터를 검증 데이터셋으로 사용  

In [1]:
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar

--2024-04-05 06:43:26--  http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar [following]
--2024-04-05 06:43:26--  https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  42.5MB/s    in 11s     

2024-04-05 06:44:04 (41.3 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]



In [ ]:
!wget http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar

--2024-04-05 08:04:08--  http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar [following]
--2024-04-05 08:04:09--  https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007  93%[=================>  ] 400.49M  16.4MB/s    eta 3s     

```
pascal_datasets
pascal_datasets/trainval
pascal_datasets/test
pascal_datasets/VOC
pascal_datasets/VOC/images
pascal_datasets/VOC/labels
pascal_datasets/VOC/images/train2007
pascal_datasets/VOC/images/val2007
pascal_datasets/VOC/images/test2007
pascal_datasets/VOC/labels/train2007
pascal_datasets/VOC/labels/val2007
pascal_datasets/VOC/labels/test2007
```

In [ ]:
from pathlib import Path

In [ ]:
root = Path('./pascal_datasets')

In [ ]:
# parents=True로 설정되면 pascal_datasets를 생성하고 아래 trainval를 생성
# 디렉토리 구조가 없는 경우 전체 경로를 한 번에 생성
Path('./pascal_datasets/trainval').mkdir(parents=True, exist_ok=True)
Path('./pascal_datasets/test').mkdir(parents=True, exist_ok=True)

In [ ]:
'''
pascal_datasets/VOC/images
pascal_datasets/VOC/labels
pascal_datasets/VOC/images/train2007
pascal_datasets/VOC/images/val2007
pascal_datasets/VOC/images/test2007
pascal_datasets/VOC/labels/train2007
pascal_datasets/VOC/labels/val2007
pascal_datasets/VOC/labels/test2007
경로를 한번에 구조를 만드는 알고리즘을 작성해보자.
(단, 2중 for문을 사용)
'''
for path1 in ('images', 'labels'):
    for path2 in ('train2007', 'val2007', 'test2007'):
        new_path = root / 'VOC' / path1 / path2
        new_path.mkdir(parents=True, exist_ok=True)

In [ ]:
!tar -xvf VOCtrainval_06-Nov-2007.tar -C ./pascal_datasets/trainval/
!tar -xvf VOCtest_06-Nov-2007.tar -C ./pascal_datasets/test/

#**3. YOLO 포멧으로 변경하기**
* xml에서 좌표(XMin, YMin, XMax, YMax)를 YOLO 모델에서 사용하기 위해 포멧 변경이 필요
* YOLO 형식: (클래스번호, X의 center좌표, Y의 center좌표, 너비, 높이)

In [ ]:
!git clone https://github.com/ssaru/convert2Yolo.git

In [ ]:
%cd convert2Yolo
%pip install -qr requirements.txt

#**4. names 파일**
* 머신러닝, 딥러닝 모델이 데이터셋 내의 클래스를 인식하고 구분할 수 있도록 클래스 이름을 정의하는데 사용되는 파일 형식
* Darknet 프레임워크와 머신러닝 라이브러리와 함께 사용
* YOLO와 같은 유명한 객체 탐지 알고리즘을 구현하는데 사용

In [ ]:
# trainval 데이터 yolo 포멧 변환
!python3 example.py --datasets VOC --img_path ./pascal_datasets/trainval/VOCdevkit/VOC2007/JPEGImages/ --label /content/pascal_datasets/trainval/VOCdevkit/VOC2007/Annotations/ --convert_output_path /content/pascal_datasets/VOC/labels/train2007 --img_type ".jpg" --manifest_path /content/ --cls_list_file ./voc.names

In [ ]:
# test 데이터 yolo 포멧 변환
!python3 example.py --datasets VOC --img_path ./pascal_datasets/test/VOCdevkit/VOC2007/JPEGImages/ --label /content/pascal_datasets/test/VOCdevkit/VOC2007/Annotations/ --convert_output_path /content/pascal_datasets/VOC/labels/test2007 --img_type ".jpg" --manifest_path /content/ --cls_list_file ./voc.names

### 문제
PascalVOC 제공 파일로 train, val 라벨 분할
* /content/pascal_datasets/trainval/VOCdevkit/VOC2007/ImageSets/Main/val.txt 를 읽어서 /content/pascal_datasets/VOC/labels/train2007/ 파일을 /content/pascal_datasets/VOC/labels/val2007/ 로 옮기기

In [8]:
import shutil

In [9]:
path = '/content/pascal_datasets/trainval/VOCdevkit/VOC2007/ImageSets/Main/val.txt'

with open(path) as f:
    image_ids = f.read().strip().split()
    for id in image_ids:
        ori_path = '/content/pascal_datasets/VOC/labels/train2007/'
        mv_path = '/content/pascal_datasets/VOC/labels/val2007/'
        shutil.move(f'{ori_path}/{id}.txt', f'{mv_path}/{id}.txt')
print('파일이동 완료!')

FileNotFoundError: [Errno 2] No such file or directory: '/content/pascal_datasets/VOC/labels/train2007//000005.txt'

In [ ]:
# /content/pascal_datasets/trainval/VOCdevkit/VOC2007/JPEGImages
# /content/pascal_datasets/VOC/images/train2007
# /content/pascal_datasets/VOC/images/val2007

# /content/pascal_datasets/test/VOCdevkit/VOC2007/JPEGImages
# /content/pascal_datasets/VOC/images/test2007

import os

path = '/content/pascal_datasets'

for folder, subset in ('trainval', 'train2007'), ('trainval', 'val2007'), ('test', 'test2007'):
    ex_imgs_path = f'{path}/{folder}/VOCdevkit/VOC2007/JPEGImages'
    label_path = f'{path}/VOC/labels/{subset}'
    img_path = f'{path}/VOC/images/{subset}'
    print(subset, ": ", len(os.listdir(label_path)))
    for lbs_list in os.listdir(label_path):
        shutil.move(os.path.join(ex_imgs_path, lbs_list.split('.')[0]+'.jpg'),
                    os.path.join(img_path, lbs_list.split('.')[0]+'.jpg'))

train2007 :  2501
val2007 :  2510
test2007 :  4952


#**5. YOLOv5 다운로드 및 설치**

In [ ]:
%cd /content/
!git clone -b v6.2 https://github.com/ultralytics/yolov5.git

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
%cd yolov5
%pip install -qr requirements.txt
!pip install numpy==1.23.0

/content/yolov5


In [ ]:
import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v6.2-0-gd3ea0df8 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 31.8/78.2 GB disk)


#**6. WanDB를 이용한 학습 및 평가 과정 로깅**
* https://wandb.ai/site
* 머신러닝/딥러닝 개발자를 위한 종합적인 보조 도구
* 딥러닝 모델 학습할 때 학습 과정에 대해 로깅을 진행
* 손실값의 감소하는 형태를 쉽게 파악할 수 있음
* 팀 단위로 실험 결과를 추적할 수 있도록 해주기 때문에 웹에서 편리하게 분석이 가능

In [ ]:
%pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


In [ ]:
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import random
import numpy as np

In [ ]:
seed = 2024
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

if deterministic:
    # 학습 및 추론 과정에서 재현성을 보장하기 위해 사용
    torch.backends.cudnn.deterministic = True
    # 모델의 학습 및 추론 성능을 최적화하기 위해 사용
    # True: 최적화된 알고리즘을 동적으로 선택
    # False: 오버헤드는 발생하지 않으나 성능이 떨어짐
    torch.backends.cudnn.benchmark = True

#**7. 어노테이션**
* 어노테이션(주석)은 데이터 집합의 데이터에 정보 또는 레이블을 추가하는 프로세스
* 컴퓨터 비전에서 이미지에서 특정 객체의 존재, 객체의 속성(색상, 크기, 모양)을 나타내는 레이블
* 수동으로 추가하거나 컴퓨터 알고리즘을 사용하여 자동으로 생성

### 7-1. 어노테이션의 종류
* Bounding Box
    * 이미지 내에서 객체의 위치 및 크기를 정의하는 작업
    * 객체 주위에 Box를 그리고 클래스(사람, 자동차)로 Label을 지정
    * Object Detection에 일반적으로 사용
    * 이미지 내에서 객체의 위치를 정의하는 간단하고 효과적인 방법
* KeyPoint
    * 객체 내의 특정 관심 지점을 표시하는 작업
    * 사람 또는 동물의 이미지 내 관절의 위치를 예측하는 것이 목표인 Task에 종종 사용
* Segmentation
    * 이미지 내에서 객체의 경계를 정의
    * 객체의 경계를 표시하고 인식하고 이미지 내의 객체를 분류하도록 학습
    * 복잡한 형태의 어노테이션이지만 이미지 객체에 대한 더 자세한 정보를 모델에게 제공하므로 성능이 향상

### 7-2. 어노테이션 방법
* 수동 주석
    * 마우스 또는 스타일러스와 같은 도구를 사용하여 이미지 내의 각 객체에 수동으로 레이블을 지정하는 작업
* 자동 주석
    * 컴퓨터 알고리즘을 사용하여 이미지 내의 객체에 자동으로 레이블을 지정하는 작업
    * 가장 빠르지만 정확도가 낮음
    * 자동 주석은 수작업 비용이 높은 대규모 데이터셋에 주석을 추가하는데 사용
* 반자동 주석
    * 컴퓨터 지원 도구를 사용하여 주석 프로세스의 속도를 높이는 작업
    * 어노테이터는 도구를 사용하여 객체 주위에 경계 상자를 그릴 수 있으며, 컴퓨터는 자동으로 객체에 해당 클래스로 레이블을 지정
    * 수동 주석보다는 빠르지만, 정확성을 보장하려면 사람의 입력과 노력이 필요함

#**8. data.yaml 파일**
* custom_voc: Pascal voc 2007 데이터 명시 파일
* custom_dataset: 직접 라벨링한 데스트 데이터 명시 파일
* 파일 경로: /content/yolov/data/

#**9. YOLO v5 가중치 파일**
* yolov5s.pt
  * yolov5에 가장 작은 버전으로 경량화된 모델
  * 작은 크기의 객체를 감지하거나 시스템 리소스가 제한된 환경에서 사용
* yolov5m.pt
  * 중간 크기의 모델로, 기본적인 객체 탐지와 분류에 적합
* yolob5l.pt
  * 큰 모델로 더 높은 정확도를 제공
  * 크기가 큰 객체나 복잡한 시나리오에 유용
* yolov5x,pt
  * 가장 큰 모델로, 가장 높은 정확도를 목표로 함

In [ ]:
%cd /content/yolov5
!python train.py --img 640 --batch 32 --epochs 10 --data custom_voc.yaml --weights yolov5s.pt --seed 2024

/content/yolov5
2024-04-02 23:14:42.782708: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 23:14:42.782759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 23:14:42.784189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 23:14:43.990791: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: rhxocm. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=custom_voc.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=640, rect=False, r

In [ ]:
# Pascal VOC 테스트 데이터로 테스
# half: 신경망 모델의 계산을 16비트 부동 소수점 형식으로 처리으로써 메모리 사용량을 줄이고 계산 속도를 향상
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data custom_voc.yaml --img 640 --iou 0.4 --task test --half

Traceback (most recent call last):
  File "/content/yolov5/val.py", line 395, in <module>
    opt = parse_opt()
  File "/content/yolov5/val.py", line 356, in parse_opt
    opt.data = check_yaml(opt.data)  # check YAML
  File "/content/yolov5/utils/general.py", line 427, in check_yaml
    return check_file(file, suffix)
  File "/content/yolov5/utils/general.py", line 453, in check_file
    assert len(files), f'File not found: {file}'  # assert file was found
AssertionError: File not found: custom_voc.yaml


In [ ]:
# 직접 라벨링한 데스트 데이터로 테스트
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data custom_dataset.yaml --img 640 --iou 0.4 --task test --half

Traceback (most recent call last):
  File "/content/yolov5/val.py", line 395, in <module>
    opt = parse_opt()
  File "/content/yolov5/val.py", line 356, in parse_opt
    opt.data = check_yaml(opt.data)  # check YAML
  File "/content/yolov5/utils/general.py", line 427, in check_yaml
    return check_file(file, suffix)
  File "/content/yolov5/utils/general.py", line 453, in check_file
    assert len(files), f'File not found: {file}'  # assert file was found
AssertionError: File not found: custom_dataset.yaml


In [ ]:
# 이미지들에 대한 경계상자 이미지 생성
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source /content/pascal_datasets/VOC/custom_datasets/obj_train_data

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/pascal_datasets/VOC/custom_datasets/obj_train_data, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.2-0-gd3ea0df8 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 257, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 252, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5/detect.py", line 93, in run
    model = DetectMul